# Parsing unique items and item modifiers in map change data
Natalia Vélez, June 2020

Some items have special modifiers attached to the object ID:

```
483.81 -5123 -1390 2742u3 2276909
…
511.16 -5123 -1392 2742u2 2276909
…
516.87 -5123 -1392 2742u1 2276905
…
522.20 -5123 -1392 2742u0 2276914
```

That exact ID “2742u3” doesn’t match any objects, but it’s possible that “u[0-9]” is a counter on how many uses are remaining in an object with multiple uses. Object 2742 is a carrot pile, which has up to 5 uses. This log could show that multiple players went up to this carrot pile and ate from it.

How many modifiers are there, and what do they mean?

In [1]:
import os, re, glob, datetime
from os.path import join as opj
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

Maplog files:

In [2]:
map_files = glob.glob('outputs/maplogs/maplog*.tsv')
map_files.sort()

print(*map_files[:5], sep='\n')

outputs/maplogs/maplog_release-283_start-1573895672.tsv
outputs/maplogs/maplog_release-283_start-1573982073.tsv
outputs/maplogs/maplog_release-286_start-1574068473.tsv
outputs/maplogs/maplog_release-286_start-1574102503.tsv
outputs/maplogs/maplog_release-286_start-1574151678.tsv


Helper: Process individual file

In [19]:
file = map_files[3]

# Trim invalid 
map_df = pd.read_csv(file, sep='\t')
map_df = map_df[map_df.player_id > 0]
map_df.head()

# Unique items
obj_list = map_df.object_id.tolist()
unique_obj = np.unique(obj_list).tolist()
unique_obj.sort()

# Unique modifiers
unique_str = '|'.join(unique_obj)
item_modifiers = re.findall('[a-z]+', unique_str)
unique_modifiers = np.unique(item_modifiers).tolist()
unique_modifiers.sort()

In [17]:
re.findall('[0-9]+v[0-9]+', unique_str)

['912v0',
 '912v1',
 '912v2',
 '912v3',
 '912v4',
 '912v5',
 '912v6',
 '917v6',
 '990v6']

In [7]:
unique_str = '|'.join([str(i) for i in unique_items])

item_modifiers = re.findall(unique_str, 'f[0-9]+')
item_modifiers 

NameError: name 'unique_items' is not defined

In [20]:
map_df[map_df.object_id.str.len() == 9]

,t_elapsed,x,y,object_id,player_id
322558,19737.65,39,-513,146410607,2287310
322575,19738.66,39,-513,146410607,2287310
322611,19740.06,39,-513,146410607,2287310
322692,19744.84,39,-513,146410607,2287467
322717,19746.34,39,-513,146410607,2287467
...,...,...,...,...,...
707014,44943.53,248,-491,148112975,2288593
707036,44946.12,-254,35,148900975,2288595
707154,44960.09,-254,36,148900975,2288604
707645,45016.85,-254,37,148900975,2288581
